Final Project: The correlation of a country’s economy on CO₂ Emission

In [5]:
import pandas as pd
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objects as go
import pycountry

all_country_data = []
# get emission by sector
df_sector = pd.read_csv('per-capita-co2-sector.csv', sep=',')
df = pd.read_csv('CO2_Emissions_1960-2018.csv')
df_gdp = pd.read_csv('countries_gdp_hist.csv', sep=';')

# reshape the data to be able to display CO2 emissions per year
df = df.melt(id_vars=["Country Name"], var_name="Year", value_name="Emission Per Year")
df['Year'] = df['Year'].astype(int)

# convert country names to ISO format
def get_iso_alpha_3(country_name):
    try:
        return pycountry.countries.lookup(country_name).alpha_3
    except LookupError:
        return None

df['iso_alpha'] = df['Country Name'].apply(lambda x: "OWID_WRL" if x == "World" else get_iso_alpha_3(x))

df_merged = df.merge(df_sector, left_on=['iso_alpha', 'Year'], right_on=['Code', 'Year'], how='right')
df_merged['Total'] = df_merged.iloc[:, 6:].sum(axis=1, skipna=True)

# Process GDP data
df_gdp = df_gdp.rename(columns={"country_code": "iso_alpha", "year": "Year", "total_gdp": "GDP"})

# init Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("CO2 Emission per Capita per Year", style={"text-align": "center"}),
    # create world map
    dcc.Graph(
        id='choropleth-map',
        figure=px.choropleth(
            df_merged,
            locations="iso_alpha",
            color="Emission Per Year",
            scope="world",
            animation_frame="Year",
            hover_name="Country Name",
            width=1000,
            height=600,
            color_continuous_scale=["green", "yellow", "red"],
            range_color=(0, 25),
            custom_data=["Year"]
        )
    ),
    html.H2("CO2 Emission by Sector Comparison", style={"text-align": "center"}),
    dcc.Graph(id='sunburst-chart'),
    html.H2("Country GDP Comparison", style={"text-align": "center"}),
    html.Div(id='dollar-sign-container', style={'display': 'flex', 'flex-wrap': 'wrap'})
])

@app.callback(
    [
        Output('sunburst-chart', 'figure'),
        Output('dollar-sign-container', 'children')
    ],
    [Input('choropleth-map', 'clickData')]
)
def update_visuals(clickData):
    # Default: Show World for year 1990
    default_country_iso = 'OWID_WRL'
    default_year = 1990

    global all_country_data
    if 'all_country_data' not in globals():
        all_country_data = []

    if clickData is None:
        country_iso = default_country_iso
        selected_year = default_year
    else:
        country_iso = clickData['points'][0]['location']
        selected_year = clickData['points'][0]['customdata'][0]

    if selected_year is None:
        selected_year = default_year

    existing_entry = next((entry for entry in all_country_data if entry["iso"] == country_iso and entry["year"] == selected_year), None)

    if existing_entry:
        all_country_data.remove(existing_entry)
    else:
        country_data = df_merged[(df_merged['iso_alpha'] == country_iso) & (df_merged['Year'] == selected_year)]
        if not country_data.empty:
            # add country to comparison
            all_country_data.append({
                "iso": country_iso,
                "year": selected_year,
                "data": country_data.iloc[0]
            })

    # Sunburst data creation
    sector_columns = [        
        "buildings", "electricity and heat", "energy production", "industry", "bunker fuels", 
        "land use change and forestry", "transport", "manufacturing and construction", "other fuel combustion"]

    color_map = {
        "buildings": "#636EFA",
        "electricity and heat": "#EF553B",
        "energy production": "#00CC96",
        "industry": "#AB63FA",
        "bunker fuels": "#FFA15A",
        "land use change and forestry": "#19D3F3",
        "transport": "#FF6692",
        "manufacturing and construction": "#B6E880",
        "other fuel combustion": "#FF97FF",
    }
    
    labels = []
    parents = []
    values = []
    colors = []

    # loop through all sectors
    for sector in sector_columns:
        labels.append(sector)
        parents.append(" ")
        values.append(0)
        colors.append(color_map.get(sector, "#CCCCCC"))

    # Add values for sector per country + year
    for entry in all_country_data:
        country_iso = entry["iso"]
        year = entry["year"]
        sector_data = entry["data"]

        for sector in sector_columns:
            value = sector_data.get(sector, 0)
            if value > 0:
                labels.append(f"{sector}|{country_iso}")
                parents.append(sector)
                values.append(value)
                colors.append(color_map.get(sector, "#CCCCCC"))  # set same color as parent

                sector_index = sector_columns.index(sector)
                values[sector_index] += value

    sunburst_chart = go.Figure(go.Sunburst(
        labels=labels,
        parents=parents,
        values=values,
        marker=dict(colors=colors)
    ))

    # Generate dollar sign icons for selected countries
    if all_country_data:
        selected_years = [entry["year"] for entry in all_country_data]
        max_gdp = df_gdp[(df_gdp['Year'].isin(selected_years)) & (df_gdp['GDP'] > 0)]['GDP'].max()
        min_gdp = df_gdp[(df_gdp['Year'].isin(selected_years)) & (df_gdp['GDP'] > 0)]['GDP'].min()

        dollar_signs = []
        for entry in all_country_data:
            country_iso = entry["iso"]
            year = entry["year"]

            country_gdp = df_gdp[(df_gdp['iso_alpha'] == country_iso) & (df_gdp['Year'] == year)]["GDP"].values
            if len(country_gdp) == 0 or country_gdp[0] == 0:
                size = 50  # Minimum size for missing or zero GDP
            else:
                gdp_value = country_gdp[0]
                size = 50 + (450 * (gdp_value - min_gdp) / (max_gdp - min_gdp))  # Scale size from 50 to 500

            dollar_signs.append(
                html.Div(
                    children=[
                        html.Img(
                            src="/assets/dollar_sign.svg",
                            style={"width": f"{size}px", "height": f"{size}px", "margin": "5px"}
                        ),
                        html.Div(country_iso, style={"text-align": "center", "font-size": "12px"})
                    ],
                    style={"display": "flex", "flex-direction": "column", "align-items": "center"}
                )
            )
    else:
        dollar_signs = []

    return sunburst_chart, dollar_signs

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)
